In [1]:
import time, os, socket, threading
from urllib.request import urlretrieve
from urllib.error import HTTPError, URLError

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import \
ElementClickInterceptedException, NoSuchElementException, ElementNotInteractableException
import cv2

In [2]:
# chorme_options = webdriver.ChromeOptions()
# chorme_options.add_argument('--headless')
# chorme_options.add_argument('--no-sandbox')
# chorme_options.add_argument('--disable-dev-shm-usage')
# webbrowser를 띄우지 않을때

In [3]:
#search = "coffee" # 검색 키워드
chromedriver_path = './chromedriver/chromedriver' # 드라이버 파일 위치

# Crawling Func

In [4]:
def create_folder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
        
def scroll_down() :
    scroll_count = 0
    print("[scroll_down() : 스크롤 다운 시작]")
    
    last_height = driver.execute_script('return document.body.scrollHeight')
    after_click = False
    after_click_exception_span = False
    while True :
        print(f"[스크롤 다운: {scroll_count}]")
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        scroll_count += 1
        time.sleep(1)
        
        new_height = driver.execute_script('return document.body.scrollHeight')
        
        if last_height == new_height :
            try : 
                if driver.find_element('xpath', '//*[@id="islmp"]/div/div/div/div[2]/span') != 0 :
                    more_span = driver.find_element('xpath', '//*[@id="islmp"]/div/div/div/div[2]/span')
                    more_span.click()
                    print('고약한 구글')
            except NoSuchElementException as e :
                print(e)
                
            if after_click is True :
                break
            else :
                try :
                    more_button = driver.find_element('xpath', '//*[@id="islmp"]/div/div/div/div[1]/div[2]/div[2]/input')
                    if more_button.is_displayed():
                        more_button.send_keys(Keys.ENTER)
                        after_click = True
                    
                except NoSuchElementException as e:
                    # scroll button이 없을때
                    print(e)
                    break
        last_height = new_height
        
def click_and_save(dir_name, index, img, img_list_length):
    global scraped_count
    
    try:
        img.click()
        driver.implicitly_wait(5) # 5초 기다림
        xpath = '//*[@id="Sva75c"]/div/div/div[3]/div[2]/c-wiz/div/div[1]/div[1]/div[3]/div/a/img'
        src = driver.find_element('xpath', xpath).get_attribute('src')
#         src = driver.find_element(By.CSS_SELECTOR, '.n3VNCb.KAlRDb').get_attribute('src')
        if src.split('.')[-1] == 'png':
            urlretrieve(src, dir_name+'/'+str(scraped_count+1)+".png")
            print(f"{index+1}/{img_list_length} PNG 이미지 저장")
        else :
            urlretrieve(src, dir_name+'/'+str(scraped_count+1)+".jpg")
            print(f"{index+1}/{img_list_length} JPG 이미지 저장")

        scraped_count += 1
    except HTTPError as e :
        print(e)
    except Exception as e :
        print(e)
        return
    
def scraping(dir_name, query):
    global scraped_count

    url = f"https://www.google.com/search?q={query}&tbm=isch&ved=2ahUKEwi037baoc76AhVEyIsBHQnWDywQ2-cCegQIABAA&oq=f&gs_lcp=CgNpbWcQARgAMgQIIxAnMgQIIxAnMggIABCABBCxAzIFCAAQgAQyCAgAEIAEELEDMgUIABCABDIICAAQgAQQsQMyBQgAEIAEMgUIABCABDIFCAAQgARQxwJYxwJggxJoAHAAeACAAYYBiAHhAZIBAzEuMZgBAKABAaoBC2d3cy13aXotaW1nwAEB&sclient=img&ei=4C5AY7TvMMSQr7wPiay_4AI&bih=969&biw=1920&rlz=1C5CHFA_enKR1026KR1026#imgrc=X6d1rGBwHMhXsM"
    driver.get(url)
    driver.maximize_window()
    

    scroll_down()
    print('[스크롤 완료]')
    driver.execute_script("window.scrollTo(0, 0)")
    
    div = driver.find_element('xpath', '//*[@id="islrg"]/div[1]')
    img_list = div.find_elements(By.CSS_SELECTOR, ".rg_i.Q4LuWd")
#     img_list = div.find_elements('xpath', '//*[@id="islrg"]/div[1]/div[1]/a[1]/div[1]/img')
    print(len(img_list))
    img_list_length = len(img_list)
    #######################################################################################

    for index, img in enumerate(img_list) :
        try:
            click_and_save(dir_name, index, img, img_list_length)
        except ElementClickInterceptedException as e :
            print(e)
            driver.execute_script("window.scrollTo(0, window.scrollY+100)")
            click_and_save(dir_name, index, img, img_list_length)
        except NoSuchElementException as e :
            print(e)
            driver.execute_script("window.scrollTo(0, window.scrollY+100)")
            click_and_save(dir_name, index, img, img_list_length)
        except ConnectionResetError as e :
            print(e)
            pass
        except URLError as e :
            print(e)
            pass
        except socket.timeout as e :
            print(e)
            pass
        except socket.gaierror as e :
            print(e)
            pass
        except ElementNotInteractableException as e:
            print(e)
            break
            
    try :
        print("[스크래핑 종료 (성공률: {:.2f})]".format((scraped_count / img_list_length) * 100))
    except ZeroDivisionError as e :
        print(e)
    
    driver.quit()

# Preprocess Func

In [5]:
def resize_and_remove(dir_name, query, filter_size):
    # 일정 해상도 이하이거나 손상된 이미지 제거
    filtered_count = 0
    
    for index, file_name in enumerate(os.listdir(dir_name)):
        try :
            file_path = os.path.join(dir_name, file_name)
            img = cv2.imread(file_path)
            height = img.shape[0]
            width = img.shape[1]
            if height >= width and width > 512 :
                img = cv2.resize(img, (512, int(512/height*width)), interpolation=cv2.INTER_AREA)
                cv2.imwrite(file_path, img)
                print(file_path, ' resized')
            elif width > height and height > 512 :
                img = cv2.resize(img, dsize=(int(512/width*height), 512), interpolation=cv2.INTER_AREA)
                cv2.imwrite(file_path, img)
                print(file_path, ' resized')
                
        except OSError as e :
            print(e)
            os.remove(file_path)
            filtered_count += 1 
        except AttributeError as e:
            print(e)
            os.remove(file_path)
            filtered_count += 1 
        except TypeError as e:
            print(e)
            print(img.shape)
    print(f"[이미지 제거 개수: {filtered_count}/{scraped_count}]")

In [6]:
query_list = ['flooding', 'flooding road', '강남역 침수','침수', 
              '道路浸水', '車の洪水', # 중국어, 일본어 (도로침수, 차 침수)
             'Überschwemmungen auf der Straße', # 독일어, 도로 침수
             'inondation des routes', # 프랑스어, 도로 침수
              'inundación de coches', # 스페인어, 차 침수
              'flooded', '침수 cctv',
              
              ### 자동차 포커스 키워드
              'flooded car',
              'überflutetes Auto', # 독일어
             '被淹的车', #중국어
             'coche inundado', # 스페인어
             'voiture inondée', # 프랑스어
             ]
              

In [7]:
socket.setdefaulttimeout(30)
driver = webdriver.Chrome(chromedriver_path)

scraped_count = 0

path = './data/'
#query = input('검색어 입력: ')
query = '被淹的车'
dir_name = path+query
create_folder(dir_name)
print(f"[{dir_name} 디렉토리 생성]")

scraping(dir_name, query)
print('##################### 크롤링 끝 #############################')
resize_and_remove(dir_name, query, 512)

/var/folders/63/8nyxhd_90vg4rctvs_pfm8440000gn/T/ipykernel_4360/4117451573.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver_path)


[./data/被淹的车 디렉토리 생성]
[scroll_down() : 스크롤 다운 시작]
[스크롤 다운: 0]
[스크롤 다운: 1]
[스크롤 다운: 2]
[스크롤 다운: 3]
[스크롤 다운: 4]
[스크롤 다운: 5]
[스크롤 다운: 6]
[스크롤 다운: 7]
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="islmp"]/div/div/div/div[2]/span"}
  (Session info: chrome=106.0.5249.103)
Stacktrace:
0   chromedriver                        0x0000000100d5af44 chromedriver + 3813188
1   chromedriver                        0x0000000100cecae0 chromedriver + 3361504
2   chromedriver                        0x00000001009f0c5c chromedriver + 232540
3   chromedriver                        0x0000000100a22380 chromedriver + 435072
4   chromedriver                        0x0000000100a4ff44 chromedriver + 622404
5   chromedriver                        0x0000000100a17358 chromedriver + 389976
6   chromedriver                        0x0000000100a18284 chromedriver + 393860
7   chromedriver                        0x0000000100d2ee18 chromedriver + 3632664
8   chromedriver          

265/828 JPG 이미지 저장
266/828 JPG 이미지 저장
267/828 JPG 이미지 저장
268/828 JPG 이미지 저장
269/828 JPG 이미지 저장
270/828 JPG 이미지 저장
271/828 JPG 이미지 저장
272/828 JPG 이미지 저장
273/828 JPG 이미지 저장
274/828 JPG 이미지 저장
275/828 JPG 이미지 저장
276/828 JPG 이미지 저장
277/828 JPG 이미지 저장
278/828 JPG 이미지 저장
279/828 JPG 이미지 저장
280/828 JPG 이미지 저장
281/828 JPG 이미지 저장
282/828 JPG 이미지 저장
283/828 JPG 이미지 저장
284/828 JPG 이미지 저장
285/828 JPG 이미지 저장
286/828 JPG 이미지 저장
287/828 JPG 이미지 저장
288/828 JPG 이미지 저장
289/828 JPG 이미지 저장
290/828 JPG 이미지 저장
291/828 JPG 이미지 저장
292/828 JPG 이미지 저장
293/828 JPG 이미지 저장
294/828 JPG 이미지 저장
295/828 JPG 이미지 저장
296/828 JPG 이미지 저장
297/828 JPG 이미지 저장
298/828 JPG 이미지 저장
299/828 JPG 이미지 저장
300/828 JPG 이미지 저장
301/828 JPG 이미지 저장
302/828 JPG 이미지 저장
303/828 JPG 이미지 저장
304/828 JPG 이미지 저장
305/828 JPG 이미지 저장
306/828 JPG 이미지 저장
307/828 JPG 이미지 저장
308/828 JPG 이미지 저장
309/828 JPG 이미지 저장
310/828 JPG 이미지 저장
311/828 JPG 이미지 저장
312/828 JPG 이미지 저장
313/828 JPG 이미지 저장
314/828 JPG 이미지 저장
315/828 JPG 이미지 저장
316/828 JPG 이미지 저장
317/828 JPG 

697/828 JPG 이미지 저장
698/828 JPG 이미지 저장
699/828 JPG 이미지 저장
700/828 JPG 이미지 저장
701/828 JPG 이미지 저장
702/828 JPG 이미지 저장
703/828 JPG 이미지 저장
704/828 JPG 이미지 저장
705/828 JPG 이미지 저장
706/828 JPG 이미지 저장
707/828 JPG 이미지 저장
708/828 JPG 이미지 저장
709/828 JPG 이미지 저장
710/828 JPG 이미지 저장
711/828 JPG 이미지 저장
712/828 JPG 이미지 저장
713/828 JPG 이미지 저장
714/828 JPG 이미지 저장
715/828 JPG 이미지 저장
716/828 JPG 이미지 저장
717/828 JPG 이미지 저장
718/828 JPG 이미지 저장
719/828 JPG 이미지 저장
720/828 JPG 이미지 저장
721/828 JPG 이미지 저장
722/828 JPG 이미지 저장
723/828 JPG 이미지 저장
724/828 JPG 이미지 저장
725/828 JPG 이미지 저장
726/828 JPG 이미지 저장
727/828 JPG 이미지 저장
728/828 JPG 이미지 저장
729/828 JPG 이미지 저장
730/828 JPG 이미지 저장
731/828 JPG 이미지 저장
732/828 JPG 이미지 저장
733/828 JPG 이미지 저장
734/828 JPG 이미지 저장
735/828 JPG 이미지 저장
736/828 JPG 이미지 저장
737/828 JPG 이미지 저장
738/828 JPG 이미지 저장
739/828 JPG 이미지 저장
740/828 JPG 이미지 저장
741/828 JPG 이미지 저장
742/828 JPG 이미지 저장
743/828 JPG 이미지 저장
744/828 JPG 이미지 저장
745/828 JPG 이미지 저장
746/828 JPG 이미지 저장
747/828 JPG 이미지 저장
748/828 JPG 이미지 저장
749/828 JPG 

In [8]:
num_img = 0
folders = os.listdir(path)

for folder in folders :
    if folder == '.DS_Store' :
        continue
    folder_path = os.path.join(path, folder)
    print('{} : {}'.format(folder, len(os.listdir(folder_path))))
    num_img += len(os.listdir(folder_path))
print('total : ', num_img)

inondation des routes : 765
침수 cctv : 698
車の洪水 : 891
침수 : 831
überflutetes Auto : 174
flooding road : 588
Überschwemmungen auf der Straße : 889
flooded car : 552
道路浸水 : 835
flooded : 817
flooding : 709
강남역 침수 : 495
inundación de coches : 876
被淹的车 : 828
total :  9948
